In [2]:
from database_cloudant import CloudantDB
from cloudant.design_document import DesignDocument
from cloudant.view import View
import sys



#db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
#db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
#db3 = CloudantDB("tweets_raw")         # raw tweet data in the specified locations
db4=CloddantDB("")
#print(len(db1.get_data()))
#print(db2.get_data(n=10000))
#print(len(db3.get_data(n=10)))



    
def dataofday(date):    
    # get all data from a specific day
    db1 = CloudantDB("tweets_keywords")    # tweets in the specified location with keywords 
    db2 = CloudantDB("tweets_no_keywords") # tweets in the sepcified locations without keyowrds
    n = 100  # number of tweets need to retrieve
    ddoc = DesignDocument(db2.curDB, '_design/ddoc001')
    ddoc.fetch()
    view = View(ddoc,"view_time")
    data = []
    for row in view(group=False,limit=n,reduce=False,key=date,include_docs=True)["rows"]:
        data.append(row)
    
    db1.close()
    db2.close()
    return data

    
    

    
#db3.close()
def city_daily_popular_10words(tweetlist):
    city=[]
    for k in tweetlist:
        if k["zone"] not in city:
            city.append(k["zone"])
    alltext={}
    for c in city:
        alltext[c]=[]
    for k in tweetlist:
        for c in city:
            if k["zone"] in c:
                alltext[c].append(k["text"])
    popdict={}             
    for c in city:
        popdict[c]=top_words(alltext[c])
    return popdict


if __name__ == "main":
    ipaddress=sys.argv[-1]

    date = "2021-01-01"
    data=dataofday(date)

    citytopwords=city_daily_popular_10words(data)
    print(citytopwords)


ModuleNotFoundError: No module named 'database_cloudant'